In [ ]:
import pandas as pd

df_inscriptions = pd.read_csv('cyprus-inscriptions-nettoyees.csv')
df_inscriptions.head()

,Inscription #,Type,Place,Date,Nature,Material,Place_Region,Place_precision,Site,Precision_Date,Date_Precision_2
0,ICS #456: p.389,CS?,Cyrenaica,NaN,No interpretation: ancient (archaizing) imitat...,Serpentine scarab.,Cyrenaica,NaN,NaN,NaN,NaN
1,ICS #419: p.373,CS,Egypt: Abydos \r(temple of Ramses II),Classical,PN: Timos \rson of Thursiya.,Stone wall. \rGraffito.,Egypt,temple of Ramses II,Abydos,early 4th,NaN
2,ICS #392: pp.363-4,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN: Menokretes \rfrom Salamis.,Stone wall.\rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN
3,ICS #388: p.362,EC?,Egypt: Abydos \r(temple of Seti I),Classical,PN: an Eteo-Cypriot?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN
4,ICS #391: p.363,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN


In [ ]:
print(df_inscriptions['Site'].value_counts().head(20))

Site
Amathus         494
Kouklia         468
Kafizin         340
Salamis         306
Larnaca         237
Kourion         230
Rantidi         173
Enkomi          128
Polis           128
Paphos           77
Athienou         76
Karnak           61
Chytroi          58
Soloi            56
Limassol         51
Abydos           46
Dhali            32
Rhizokarpaso     28
Amargetti        24
Lapithos         23
Name: count, dtype: int64


In [ ]:
norm_dict = {
    'Amathus': 'Amathous',
    'Kouklia': 'Paphos',
    'Larnaka': 'Kition',
    'Larnaca': 'Kition',
    'Dhali': 'Idalion',
    'Lapithos': 'Lapethos',
    'Polis': 'Marion',
    'Athienou': 'Golgoi',
    'Rhizokarpaso': 'Karpasia',
    'Amargetti': 'Apollo Melanthios',
    'Kalavasos': 'Kopetra',
    'Tremetousha': 'Temithus',



}

df_inscriptions['Site'] = df_inscriptions['Site'].replace(norm_dict)

In [ ]:

df_pleiades = pd.read_csv('names.csv')

df_pleiades['romanized_form_1'] = df_pleiades['romanized_form_1'].str.strip()

df_inscriptions['id_pleiades'] = None

for index, row in df_inscriptions.iterrows():
    site = row['Site']
    match = df_pleiades[df_pleiades['romanized_form_1'] == site]

    if not match.empty:
        df_inscriptions.at[index, 'id_pleiades'] = match['place_id'].values[0]

df_inscriptions.to_csv('cyprus-inscriptions-with-pleiades.csv', index=False)

print(f"Nombre d'inscriptions liées à Pleiades : {df_inscriptions['id_pleiades'].notna().sum()}")

Nombre d'inscriptions liées à Pleiades : 2647


In [ ]:
counts = df_inscriptions['id_pleiades'].value_counts().reset_index()
counts.columns = ['place_id', 'count']
df_classement = pd.merge(
    counts,
    df_pleiades[['place_id', 'romanized_form_1']],
    on='place_id',
    how='left'
)
print("--- Classement des sites par ID Pleiades ---")
print(df_classement[['count', 'romanized_form_1', 'place_id']].head(20))

--- Classement des sites par ID Pleiades ---
    count               romanized_form_1  place_id
0     545                     Nea Paphos    707586
1     545                         Paphos    707586
2     545                       Erythrai    707586
3     545                        Erythra    707586
4     545  Sebaste Claudia Flavia Paphos    707586
5     494                       Amathous    707462
6     494                        Cinyria    707462
7     494                          Nūria    707462
8     494                    Qarth.adašt    707462
9     494                   Qartihadasti    707462
10    494                           Nūrê    707462
11    306                        Salamis    707617
12    306                     Konstantia    707617
13    306                        Silluwa    707617
14    237                         Kition    707534
15    237                         Citium    707534
16    237                    Qarth.adašt    707534
17    237                   Qartihada

In [ ]:
# Voir les sites dont l'id_pleiades est vide, classés par fréquence
manquants = df_inscriptions[df_inscriptions['id_pleiades'].isna()]
print("--- Sites fréquents sans ID Pleiades ---")
print(manquants['Site'].value_counts().head(10))

--- Sites fréquents sans ID Pleiades ---
Site
Kafizin        340
Enkomi         128
Phasoulla       21
Kopetra         15
Temithus        14
Spitali         13
Pyla            12
Apeshia         11
Mouttayiaka     10
Klerou           9
Name: count, dtype: int64


Après tout ça, 2647 inscriptions avec un ID Pleiades / environ 3600 inscriptions.
Je propose de faire une deuxième salve avec Trismegistos (uniquement pour valeurs où il n'y a pas d'id_pleiades).

In [ ]:
df_tm = pd.read_csv('export_geo.csv')
df_tm['name_standard'] = df_tm['name_standard'].astype(str).str.strip()
df_inscriptions['trismegistos_id'] = None
df_inscriptions['coordinates'] = None
mask_no_pleiades = df_inscriptions['id_pleiades'].isna()

# Boucle de recherche
for index, row in df_inscriptions[mask_no_pleiades].iterrows():
    site_name = row['Site']

    match = df_tm[df_tm['name_standard'] == site_name]

    if not match.empty:
        df_inscriptions.at[index, 'trismegistos_id'] = match['id'].values[0]
        df_inscriptions.at[index, 'coordinates'] = match['coordinates;'].values[0]

print(f"Inscriptions complétées via Trismegistos : {df_inscriptions['trismegistos_id'].notna().sum()}")
print(df_inscriptions[df_inscriptions['trismegistos_id'].notna()][['Site', 'trismegistos_id', 'coordinates']].head())

Inscriptions complétées via Trismegistos : 269
          Site trismegistos_id           coordinates
111  Naukratis            1424  30.900306,30.592078;
112  Naukratis            1424  30.900306,30.592078;
230  Angastina           58251                     ;
231      Arsos           37207                     ;
232      Arsos           37207                     ;


In [ ]:
coords_split = df_inscriptions['coordinates'].str.split(',', n=1, expand=True)
df_inscriptions['latitude'] = pd.to_numeric(coords_split[0], errors='coerce')
df_inscriptions['longitude'] = pd.to_numeric(coords_split[1], errors='coerce')
valides = df_inscriptions['latitude'].notna().sum()
print(f"Nombre d'inscriptions géolocalisées : {valides}")
df_inscriptions.to_csv('cyprus-inscriptions-geocoded.csv', index=False)

Nombre d'inscriptions géolocalisées : 90


In [ ]:
df_places = pd.read_csv('places.csv')
lat_map = dict(zip(df_places['id'], df_places['representative_latitude']))
lon_map = dict(zip(df_places['id'], df_places['representative_longitude']))
pleiades_lat = df_inscriptions['id_pleiades'].map(lat_map)
pleiades_lon = df_inscriptions['id_pleiades'].map(lon_map)
df_inscriptions['latitude'] = df_inscriptions['latitude'].combine_first(pleiades_lat)
df_inscriptions['longitude'] = df_inscriptions['longitude'].combine_first(pleiades_lon)
total_geocoded = df_inscriptions['latitude'].notna().sum()
print(f"Total d'inscriptions géocodées (TM + Pleiades) : {total_geocoded}")
print(df_inscriptions[['Site', 'id_pleiades', 'trismegistos_id', 'latitude', 'longitude']].head())
df_inscriptions.to_csv('cyprus-inscriptions-geo.csv', index=False)

Total d'inscriptions géocodées (TM + Pleiades) : 2732
     Site id_pleiades trismegistos_id   latitude  longitude
0     NaN        None            None        NaN        NaN
1  Abydos      756512            None  26.185099  31.917622
2  Abydos      756512            None  26.185099  31.917622
3  Abydos      756512            None  26.185099  31.917622
4  Abydos      756512            None  26.185099  31.917622


In [ ]:
df_inscriptions

,Inscription #,Type,Place,Date,Nature,Material,Place_Region,Place_precision,Site,Precision_Date,Date_Precision_2,id_pleiades,trismegistos_id,coordinates,latitude,longitude
0,ICS #456: p.389,CS?,Cyrenaica,NaN,No interpretation: ancient (archaizing) imitat...,Serpentine scarab.,Cyrenaica,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN
1,ICS #419: p.373,CS,Egypt: Abydos \r(temple of Ramses II),Classical,PN: Timos \rson of Thursiya.,Stone wall. \rGraffito.,Egypt,temple of Ramses II,Abydos,early 4th,NaN,756512,None,None,26.185099,31.917622
2,ICS #392: pp.363-4,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN: Menokretes \rfrom Salamis.,Stone wall.\rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512,None,None,26.185099,31.917622
3,ICS #388: p.362,EC?,Egypt: Abydos \r(temple of Seti I),Classical,PN: an Eteo-Cypriot?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512,None,None,26.185099,31.917622
4,ICS #391: p.363,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512,None,None,26.185099,31.917622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,Enkomi (Schaeffer) 1959 #4\r=Les cylindres-sce...,CM?,Famagusta: Enkomi,LC,1 sign?,Cylinder seal:\rbrown steatite or serpentine.,Famagusta,NaN,Enkomi,NaN,LC 3A,None,37211,;,NaN,NaN
3673,Louvre AM 1639\r=BCH 81 (1957) #9: pp.14-15,CM,Famagusta: Sinda,LC,1 sign.,Cylinder seal: hematite.,Famagusta,NaN,Sinda,NaN,early LC 3A,639111,None,None,37.120499,29.604500
3674,"Kourion B1625\r=Bamboula at Kourion, p.144\r",CM?,"Limassol: Kourion \r""Bamboula""",LC,1 sign? (secondary carving),Cylinder seal:\rsteatite.,Limassol,Bamboula,Kourion,NaN,LC 3A,707549,None,None,34.665429,32.885860
3675,"K-AD #448\r=K-AD II, p.40, fig.63:6",CM?,"Larnaca: Kalavasos ""Ayios Dhimitrios""",LC,1 sign.,Metal weight.,Larnaca,Ayios Dhimitrios,Kopetra,NaN,LC 2C,None,37224,;,NaN,NaN


In [ ]:
from geopy.geocoders import ArcGIS
from geopy.distance import geodesic
import time

geolocator = ArcGIS(timeout=10)

cyprus_center = (35.1264, 33.4299)
egypt_sites = ['Ehnasya', 'Gebel Abou Fodah', 'Gizeh', 'Nubia, Bouhen', 'Shatt El-Saba El Rigal', 'Karnak', 'Abydos', 'Thebes']

mask_no_geo = df_inscriptions['latitude'].isna() & df_inscriptions['Site'].notna()
sites_restants = df_inscriptions.loc[mask_no_geo, 'Site'].unique()

print(f"Lancement du géocodage libre pour {len(sites_restants)} sites...\n")

geo_results_final = {}

for site in sites_restants:
    try:
        search_query = site if any(e in str(site) for e in egypt_sites) else f"{site}, Cyprus"

        location = geolocator.geocode(search_query)

        if location:
            point = (location.latitude, location.longitude)
            dist = geodesic(point, cyprus_center).km

            is_egypt = any(e in str(site) for e in egypt_sites)

            if is_egypt or dist < 200:
                geo_results_final[site] = point
                print(f"Succès ArcGis : {site} > {location.address} ({int(dist)}km de Chypre)")
            else:
                print(f"Ignoré (Trop loin - {int(dist)}km) : {site} > {location.address}")
        else:
            print(f"Échec : {site}")

        time.sleep(0.4)

    except Exception as e:
        print(f"Erreur pour {site} : {e}")
        time.sleep(1)

def update_coords_final(row):
    if pd.isna(row['latitude']) and row['Site'] in geo_results_final:
        coords = geo_results_final[row['Site']]
        row['latitude'] = coords[0]
        row['longitude'] = coords[1]
    return row

df_inscriptions = df_inscriptions.apply(update_coords_final, axis=1)

Lancement du géocodage libre pour 111 sites...

Succès ArcGis : Ehnasya > Ahnasya, Markaz Ahnasya, Bani Suwayf (709km de Chypre)
Succès ArcGis : Gebel Abou Fodah > Gebel Abu Foda (890km de Chypre)
Succès ArcGis : Gizeh > Gizeh, Al Jīzah (604km de Chypre)
Succès ArcGis : Nubia, Bouhen > Nubia (11812km de Chypre)
Succès ArcGis : Shatt El-Saba El Rigal > Rigal, Lot-et-Garonne, Nouvelle-Aquitaine (2976km de Chypre)
Succès ArcGis : Policoro > Cyprus (20km de Chypre)
Succès ArcGis : Akanthou > Akanthou (34km de Chypre)
Succès ArcGis : Akhna > Akhna (33km de Chypre)
Succès ArcGis : Angastina > Angastina (16km de Chypre)
Succès ArcGis : Arsos > Arsos (18km de Chypre)
Succès ArcGis : Ayia Triada > Ayia Triada (59km de Chypre)
Succès ArcGis : Ayios Varnavas > Áyios Varnávas Monastery (41km de Chypre)
Succès ArcGis : Enkomi > Enkomi (41km de Chypre)
Succès ArcGis : Koma tou Yalou > Koma Tou Yialou (31km de Chypre)
Succès ArcGis : Magrastica > Cyprus (20km de Chypre)
Succès ArcGis : Phrenaros, nea

In [ ]:
df_inscriptions

,Inscription #,Type,Place,Date,Nature,Material,Place_Region,Place_precision,Site,Precision_Date,Date_Precision_2,id_pleiades,trismegistos_id,coordinates,latitude,longitude
0,ICS #456: p.389,CS?,Cyrenaica,NaN,No interpretation: ancient (archaizing) imitat...,Serpentine scarab.,Cyrenaica,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,ICS #419: p.373,CS,Egypt: Abydos \r(temple of Ramses II),Classical,PN: Timos \rson of Thursiya.,Stone wall. \rGraffito.,Egypt,temple of Ramses II,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
2,ICS #392: pp.363-4,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN: Menokretes \rfrom Salamis.,Stone wall.\rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
3,ICS #388: p.362,EC?,Egypt: Abydos \r(temple of Seti I),Classical,PN: an Eteo-Cypriot?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
4,ICS #391: p.363,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,Enkomi (Schaeffer) 1959 #4\r=Les cylindres-sce...,CM?,Famagusta: Enkomi,LC,1 sign?,Cylinder seal:\rbrown steatite or serpentine.,Famagusta,NaN,Enkomi,NaN,LC 3A,NaN,37211.0,;,35.162720,33.883220
3673,Louvre AM 1639\r=BCH 81 (1957) #9: pp.14-15,CM,Famagusta: Sinda,LC,1 sign.,Cylinder seal: hematite.,Famagusta,NaN,Sinda,NaN,early LC 3A,639111.0,NaN,None,37.120499,29.604500
3674,"Kourion B1625\r=Bamboula at Kourion, p.144\r",CM?,"Limassol: Kourion \r""Bamboula""",LC,1 sign? (secondary carving),Cylinder seal:\rsteatite.,Limassol,Bamboula,Kourion,NaN,LC 3A,707549.0,NaN,None,34.665429,32.885860
3675,"K-AD #448\r=K-AD II, p.40, fig.63:6",CM?,"Larnaca: Kalavasos ""Ayios Dhimitrios""",LC,1 sign.,Metal weight.,Larnaca,Ayios Dhimitrios,Kopetra,NaN,LC 2C,NaN,37224.0,;,35.264820,33.660410


In [ ]:
total_geocoded = df_inscriptions['latitude'].notna().sum()
print(f"Total d'inscriptions géocodées (Pleiades + Trismegistos + ArcGis) : {total_geocoded}")

Total d'inscriptions géocodées (Pleiades + Trismegistos + ArcGis) : 3480


In [ ]:
import numpy as np
sites_a_effacer = ['Shatt El-Saba El Rigal', 'Mari', 'Nubia, Bouhen']
df_inscriptions.loc[df_inscriptions['Site'].isin(sites_a_effacer), ['latitude', 'longitude']] = np.nan
nb_effaces = len(df_inscriptions[df_inscriptions['Site'].isin(sites_a_effacer)])
print(f"Coordonnées supprimées pour {nb_effaces} inscriptions.")

Coordonnées supprimées pour 3 inscriptions.


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster

df_valid_geo = df_inscriptions.dropna(subset=['latitude', 'longitude'])

geometry = [Point(xy) for xy in zip(df_valid_geo['longitude'], df_valid_geo['latitude'])]
gdf = gpd.GeoDataFrame(df_valid_geo, geometry=geometry, crs="EPSG:4326")

m = folium.Map(location=[35.1, 33.3], zoom_start=9, tiles='CartoDB positron')
marker_cluster = MarkerCluster().add_to(m)

couleurs_periodes = {
    'Classical': 'blue',
    'Hellenistic': 'green',
    'Roman': 'red',
    'Archaic': 'orange',
    'LC': 'purple',
    'Geometric': 'cadetblue',
    '': 'gray'
}

for _, row in gdf.iterrows():
    periode = row['Date'] if row['Date'] in couleurs_periodes else ''
    color = couleurs_periodes[periode]

    popup_text = f"""
    <div style="font-family: sans-serif;">
        <b>Language:</b> {row['Type']}<br>
        <b>Site:</b> {row['Site']}<br>
        <b>Nature:</b> {row['Nature']}<br>
        <b>Période:</b> {row['Date'] if row['Date'] != '' else 'N/A'}<br>
        <b>Précision:</b> {row['Precision_Date']}<br>
        <b>Source Geo:</b> {'Pleiades' if pd.notna(row['id_pleiades']) else 'Autre'}
    </div>
    """

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(marker_cluster)

m.save('carte_inscriptions_chypre.html')
print(f"{len(gdf)} inscriptions projetées sur la carte.")

3387 inscriptions projetées sur la carte.


On dirait que les inscriptions de Kalavasos ont disparues

In [ ]:
df_inscriptions

,Inscription #,Type,Place,Date,Nature,Material,Place_Region,Place_precision,Site,Precision_Date,Date_Precision_2,id_pleiades,trismegistos_id,coordinates,latitude,longitude
0,ICS #456: p.389,CS?,Cyrenaica,NaN,No interpretation: ancient (archaizing) imitat...,Serpentine scarab.,Cyrenaica,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,ICS #419: p.373,CS,Egypt: Abydos \r(temple of Ramses II),Classical,PN: Timos \rson of Thursiya.,Stone wall. \rGraffito.,Egypt,temple of Ramses II,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
2,ICS #392: pp.363-4,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN: Menokretes \rfrom Salamis.,Stone wall.\rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
3,ICS #388: p.362,EC?,Egypt: Abydos \r(temple of Seti I),Classical,PN: an Eteo-Cypriot?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
4,ICS #391: p.363,CS,Egypt: Abydos \r(temple of Seti I),Classical,PN?,Stone wall. \rGraffito.,Egypt,temple of Seti I,Abydos,early 4th,NaN,756512.0,NaN,None,26.185099,31.917622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,Enkomi (Schaeffer) 1959 #4\r=Les cylindres-sce...,CM?,Famagusta: Enkomi,LC,1 sign?,Cylinder seal:\rbrown steatite or serpentine.,Famagusta,NaN,Enkomi,NaN,LC 3A,NaN,37211.0,;,35.162720,33.883220
3673,Louvre AM 1639\r=BCH 81 (1957) #9: pp.14-15,CM,Famagusta: Sinda,LC,1 sign.,Cylinder seal: hematite.,Famagusta,NaN,Sinda,NaN,early LC 3A,639111.0,NaN,None,37.120499,29.604500
3674,"Kourion B1625\r=Bamboula at Kourion, p.144\r",CM?,"Limassol: Kourion \r""Bamboula""",LC,1 sign? (secondary carving),Cylinder seal:\rsteatite.,Limassol,Bamboula,Kourion,NaN,LC 3A,707549.0,NaN,None,34.665429,32.885860
3675,"K-AD #448\r=K-AD II, p.40, fig.63:6",CM?,"Larnaca: Kalavasos ""Ayios Dhimitrios""",LC,1 sign.,Metal weight.,Larnaca,Ayios Dhimitrios,Kopetra,NaN,LC 2C,NaN,37224.0,;,35.264820,33.660410


In [ ]:
# On cherche maintenant "Kopetra" (puisque c'est le nouveau nom normalisé)
col_name = 'Place_Site' if 'Place_Site' in df_inscriptions.columns else 'Site'
kopetra_data = df_inscriptions[df_inscriptions[col_name].astype(str).str.contains('Kopetra', case=False, na=False)]

print(f"Nombre d'inscriptions pour Kopetra : {len(kopetra_data)}")
print("-" * 30)
print(kopetra_data[[col_name, 'latitude', 'longitude']].drop_duplicates())

Nombre d'inscriptions pour Kopetra : 15
------------------------------
        Site  latitude  longitude
639  Kopetra  35.26482   33.66041


In [ ]:
mask_k = df_inscriptions['Site'].str.contains('Kopetra', case=False, na=False)
df_inscriptions.loc[mask_k, ['latitude', 'longitude']] = [34.763, 33.305]
print("Périodes pour Kopetra :", df_inscriptions[mask_k]['Date'].unique())
import folium
test_map = folium.Map(location=[34.763, 33.305], zoom_start=15)
folium.Marker([34.763, 33.305], popup="Cible Kopetra").add_to(test_map)
test_map

In [ ]:
# Cellule de correction

mask_kopetra = df_inscriptions['Site'].str.contains('Kopetra', case=False, na=False)

df_inscriptions.loc[mask_kopetra, 'latitude'] = 34.7634
df_inscriptions.loc[mask_kopetra, 'longitude'] = 33.3052

print(f"Correction effectuée pour {mask_kopetra.sum()} lignes.")

print(df_inscriptions[mask_kopetra][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 15 lignes.
        Site  latitude  longitude
639  Kopetra   34.7634    33.3052


In [ ]:
m

In [ ]:
# On cherche maintenant "Vouni" (puisque c'est le nouveau nom normalisé)
col_name = 'Site' if 'Site' in df_inscriptions.columns else 'Site'
kopetra_data = df_inscriptions[df_inscriptions[col_name].astype(str).str.contains('Vouni', case=False, na=False)]

print(f"Nombre d'inscriptions pour Vouni : {len(kopetra_data)}")
print("-" * 30)
print(kopetra_data[[col_name, 'latitude', 'longitude']].drop_duplicates())

Nombre d'inscriptions pour Vouni : 6
------------------------------
       Site   latitude  longitude
2196  Vouni  35.156326        NaN


In [ ]:
# Cellule de correction

mask_vouni = df_inscriptions['Site'].str.contains('Vouni', case=False, na=False)

df_inscriptions.loc[mask_vouni, 'latitude'] = 35.1592031022
df_inscriptions.loc[mask_vouni, 'longitude'] = 32.7732662594

print(f"Correction effectuée pour {mask_vouni.sum()} lignes.")

print(df_inscriptions[mask_vouni][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 6 lignes.
       Site   latitude  longitude
2196  Vouni  35.159203  32.773266


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster

df_valid_geo = df_inscriptions.dropna(subset=['latitude', 'longitude'])

geometry = [Point(xy) for xy in zip(df_valid_geo['longitude'], df_valid_geo['latitude'])]
gdf = gpd.GeoDataFrame(df_valid_geo, geometry=geometry, crs="EPSG:4326")

m = folium.Map(location=[35.1, 33.3], zoom_start=9, tiles='CartoDB positron')
marker_cluster = MarkerCluster().add_to(m)

couleurs_periodes = {
    'Classical': 'blue',
    'Hellenistic': 'green',
    'Roman': 'red',
    'Archaic': 'orange',
    'LC': 'purple',
    'Geometric': 'cadetblue',
    '': 'gray'
}

for _, row in gdf.iterrows():
    periode = row['Date'] if row['Date'] in couleurs_periodes else ''
    color = couleurs_periodes[periode]

    popup_text = f"""
    <div style="font-family: sans-serif;">
        <b>Language:</b> {row['Type']}<br>
        <b>Site:</b> {row['Site']}<br>
        <b>Nature:</b> {row['Nature']}<br>
        <b>Période:</b> {row['Date'] if row['Date'] != '' else 'N/A'}<br>
        <b>Précision:</b> {row['Precision_Date']}<br>
        <b>Source Geo:</b> {'Pleiades' if pd.notna(row['id_pleiades']) else 'Autre'}
    </div>
    """

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(marker_cluster)

m.save('carte_inscriptions_chypre.html')
print(f"{len(gdf)} inscriptions projetées sur la carte.")
m

In [ ]:
# Voir les sites dont la latitude est vide, classés par fréquence
manquants = df_inscriptions[df_inscriptions['longitude'].isna()]
print("--- Sites fréquents sans latitude ---")
print(manquants['Site'].value_counts())

--- Sites fréquents sans latitude ---
Site
Spitali                   13
Pyla                      12
Klerou                     9
Limnia                     7
Famagusta                  6
Maroni                     4
Morphou                    4
Galini                     4
Myrtou                     4
Mathikoloni                3
Ledra                      3
Souni                      2
Naukratis                  2
Kazaphani                  2
Salamiou                   2
Meniko                     2
Mari                       1
Marathovounos              1
Nubia, Bouhen              1
Shatt El-Saba El Rigal     1
Paramali                   1
Lysi                       1
Akamas                     1
Chrysochou                 1
Name: count, dtype: int64


A partir de là, je peux passer à la correction finale

In [ ]:
# Cellule de correction
mask_spitali = df_inscriptions['Site'].str.contains('Spitali', case=False, na=False)
df_inscriptions.loc[mask_spitali, 'latitude'] = 34.76397
df_inscriptions.loc[mask_spitali, 'longitude'] = 33.00344
print(f"Correction effectuée pour {mask_spitali.sum()} lignes.")
print(df_inscriptions[mask_spitali][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 13 lignes.
         Site  latitude  longitude
1723  Spitali  34.76397   33.00344


In [ ]:
# Cellule de correction
mask_pyla = df_inscriptions['Site'].str.contains('Pyla', case=False, na=False)
df_inscriptions.loc[mask_pyla, 'latitude'] = 35.01386
df_inscriptions.loc[mask_pyla, 'longitude'] = 33.69185
print(f"Correction effectuée pour {mask_pyla.sum()} lignes.")
print(df_inscriptions[mask_pyla][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 12 lignes.
     Site  latitude  longitude
855  Pyla  35.01386   33.69185


In [ ]:
# Cellule de correction
mask_klerou = df_inscriptions['Site'].str.contains('Klerou', case=False, na=False)
df_inscriptions.loc[mask_klerou, 'latitude'] = 35.02037
df_inscriptions.loc[mask_klerou, 'longitude'] = 33.17794
print(f"Correction effectuée pour {mask_klerou.sum()} lignes.")
print(df_inscriptions[mask_klerou][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 9 lignes.
        Site  latitude  longitude
2093  Klerou  35.02037   33.17794


In [ ]:
# Cellule de correction
mask_limnia = df_inscriptions['Site'].str.contains('Limnia', case=False, na=False)
df_inscriptions.loc[mask_limnia, 'latitude'] = 35.20222
df_inscriptions.loc[mask_limnia, 'longitude'] = 33.85969
print(f"Correction effectuée pour {mask_limnia.sum()} lignes.")
print(df_inscriptions[mask_limnia][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 7 lignes.
       Site  latitude  longitude
257  Limnia  35.20222   33.85969


In [ ]:
# Cellule de correction
mask_famagusta = df_inscriptions['Site'].str.contains('Famagusta', case=False, na=False)
df_inscriptions.loc[mask_famagusta, 'latitude'] = 35.12145
df_inscriptions.loc[mask_famagusta, 'longitude'] =  33.93883
print(f"Correction effectuée pour {mask_famagusta.sum()} lignes.")
print(df_inscriptions[mask_famagusta][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 7 lignes.
          Site  latitude  longitude
247  Famagusta  35.12145   33.93883


In [ ]:
# Cellule de correction
mask_maroni = df_inscriptions['Site'].str.contains('Maroni', case=False, na=False)
df_inscriptions.loc[mask_maroni, 'latitude'] = 34.75727
df_inscriptions.loc[mask_maroni, 'longitude'] =  33.35638
print(f"Correction effectuée pour {mask_maroni.sum()} lignes.")
print(df_inscriptions[mask_maroni][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 4 lignes.
       Site  latitude  longitude
849  Maroni  34.75727   33.35638


In [ ]:
# Cellule de correction
mask_morphou = df_inscriptions['Site'].str.contains('Morphou', case=False, na=False)
df_inscriptions.loc[mask_morphou, 'latitude'] = 35.2036
df_inscriptions.loc[mask_morphou, 'longitude'] =  32.9937
print(f"Correction effectuée pour {mask_morphou.sum()} lignes.")
print(df_inscriptions[mask_morphou][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 4 lignes.
         Site  latitude  longitude
2123  Morphou   35.2036    32.9937


In [ ]:
# Cellule de correction
mask_galini = df_inscriptions['Site'].str.contains('Galini', case=False, na=False)
df_inscriptions.loc[mask_galini, 'latitude'] = 35.135762
df_inscriptions.loc[mask_galini, 'longitude'] =  32.766461
print(f"Correction effectuée pour {mask_galini.sum()} lignes.")
print(df_inscriptions[mask_galini][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 4 lignes.
        Site   latitude  longitude
1811  Galini  35.135762  32.766461


In [ ]:
# Cellule de correction
mask_myrtou = df_inscriptions['Site'].str.contains('Myrtou', case=False, na=False)
df_inscriptions.loc[mask_myrtou, 'latitude'] = 35.30910
df_inscriptions.loc[mask_myrtou, 'longitude'] =  33.07571
print(f"Correction effectuée pour {mask_myrtou.sum()} lignes.")
print(df_inscriptions[mask_myrtou][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 6 lignes.
                Site  latitude  longitude
2545  Stavros Myrtou   35.3091   33.07571


In [ ]:
# Cellule de correction
mask_marthikoloni = df_inscriptions['Site'].str.contains('Mathikoloni', case=False, na=False)
df_inscriptions.loc[mask_marthikoloni, 'latitude'] = 34.77856
df_inscriptions.loc[mask_marthikoloni, 'longitude'] =  33.04636
print(f"Correction effectuée pour {mask_marthikoloni.sum()} lignes.")
print(df_inscriptions[mask_marthikoloni][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 3 lignes.
             Site  latitude  longitude
1662  Mathikoloni  34.77856   33.04636


In [ ]:
# Cellule de correction
mask_ledra = df_inscriptions['Site'].str.contains('Ledra', case=False, na=False)
df_inscriptions.loc[mask_ledra, 'latitude'] = 35.157167
df_inscriptions.loc[mask_ledra, 'longitude'] =  33.347297
print(f"Correction effectuée pour {mask_ledra.sum()} lignes.")
print(df_inscriptions[mask_ledra][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 3 lignes.
       Site   latitude  longitude
2987  Ledra  35.157167  33.347297


In [ ]:
# Cellule de correction
mask_souni = df_inscriptions['Site'].str.contains('Souni', case=False, na=False)
df_inscriptions.loc[mask_souni, 'latitude'] = 34.7346
df_inscriptions.loc[mask_souni, 'longitude'] =  32.8824
print(f"Correction effectuée pour {mask_souni.sum()} lignes.")
print(df_inscriptions[mask_souni][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 2 lignes.
       Site  latitude  longitude
1720  Souni   34.7346    32.8824


In [ ]:
# Cellule de correction
mask_naukratis = df_inscriptions['Site'].str.contains('Naukratis', case=False, na=False)
df_inscriptions.loc[mask_naukratis, 'latitude'] = 30.900508
df_inscriptions.loc[mask_naukratis, 'longitude'] =  30.5919275
print(f"Correction effectuée pour {mask_naukratis.sum()} lignes.")
print(df_inscriptions[mask_naukratis][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 2 lignes.
          Site   latitude  longitude
111  Naukratis  30.900508  30.591928


In [ ]:
# Cellule de correction
mask_kazaphani = df_inscriptions['Site'].str.contains('Kazaphani', case=False, na=False)
df_inscriptions.loc[mask_kazaphani, 'latitude'] = 35.32045
df_inscriptions.loc[mask_kazaphani, 'longitude'] =  33.35398
print(f"Correction effectuée pour {mask_kazaphani.sum()} lignes.")
print(df_inscriptions[mask_kazaphani][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 2 lignes.
          Site  latitude  longitude
570  Kazaphani  35.32045   33.35398


In [ ]:
# Cellule de correction
mask_salamiou = df_inscriptions['Site'].str.contains('Salamiou', case=False, na=False)
df_inscriptions.loc[mask_salamiou, 'latitude'] = 34.84135
df_inscriptions.loc[mask_salamiou, 'longitude'] =  32.69008
print(f"Correction effectuée pour {mask_salamiou.sum()} lignes.")
print(df_inscriptions[mask_salamiou][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 2 lignes.
          Site  latitude  longitude
3583  Salamiou  34.84135   32.69008


In [ ]:
# Cellule de correction
mask_meniko = df_inscriptions['Site'].str.contains('Meniko', case=False, na=False)
df_inscriptions.loc[mask_meniko, 'latitude'] = 35.11190
df_inscriptions.loc[mask_meniko, 'longitude'] =  33.14567
print(f"Correction effectuée pour {mask_meniko.sum()} lignes.")
print(df_inscriptions[mask_meniko][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 2 lignes.
        Site  latitude  longitude
2115  Meniko   35.1119   33.14567


In [ ]:
# Voir les sites dont la longitude est toujours vide, classés par fréquence
manquants = df_inscriptions[df_inscriptions['longitude'].isna()]
print("--- Sites fréquents sans latitude ---")
print(manquants[['Place_Region', 'Site']].value_counts())

--- Sites fréquents sans latitude ---
Place_Region  Site                  
Egypt         Nubia, Bouhen             1
              Shatt El-Saba El Rigal    1
Famagusta     Marathovounos             1
Larnaca       Mari                      1
Limassol      Paramali                  1
Nicosia       Lysi                      1
Paphos        Akamas                    1
              Chrysochou                1
Name: count, dtype: int64


In [ ]:
# Cellule de correction
mask_marathovounos = df_inscriptions['Site'].str.contains('Marathovounos', case=False, na=False)
df_inscriptions.loc[mask_marathovounos, 'latitude'] = 35.22431
df_inscriptions.loc[mask_marathovounos, 'longitude'] =  33.61748
print(f"Correction effectuée pour {mask_marathovounos.sum()} lignes.")
print(df_inscriptions[mask_marathovounos][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 1 lignes.
              Site  latitude  longitude
265  Marathovounos  35.22431   33.61748


In [ ]:
# Cellule de correction
mask_paramali = df_inscriptions['Site'].str.contains('Paramali', case=False, na=False)
df_inscriptions.loc[mask_paramali, 'latitude'] = 34.68136
df_inscriptions.loc[mask_paramali, 'longitude'] =  32.79179
print(f"Correction effectuée pour {mask_paramali.sum()} lignes.")
print(df_inscriptions[mask_paramali][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 1 lignes.
          Site  latitude  longitude
1682  Paramali  34.68136   32.79179


In [ ]:
# Cellule de correction
mask_lysi = df_inscriptions['Site'].str.contains('Lysi', case=False, na=False)
df_inscriptions.loc[mask_lysi, 'latitude'] = 35.1090
df_inscriptions.loc[mask_lysi, 'longitude'] =  33.6827
print(f"Correction effectuée pour {mask_lysi.sum()} lignes.")
print(df_inscriptions[mask_lysi][['Site', 'latitude', 'longitude']].head(1))

Correction effectuée pour 1 lignes.
      Site  latitude  longitude
2113  Lysi    35.109    33.6827


In [ ]:
from google.colab import files
df_inscriptions.to_csv('inscriptions_chypre_geocodees.csv', index=False, encoding='utf-8-sig')
files.download('inscriptions_chypre_geocodees.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster

df_valid_geo = df_inscriptions.dropna(subset=['latitude', 'longitude'])

geometry = [Point(xy) for xy in zip(df_valid_geo['longitude'], df_valid_geo['latitude'])]
gdf = gpd.GeoDataFrame(df_valid_geo, geometry=geometry, crs="EPSG:4326")

m = folium.Map(location=[35.1, 33.3], zoom_start=9, tiles='CartoDB positron')
marker_cluster = MarkerCluster().add_to(m)

couleurs_periodes = {
    'Classical': 'blue',
    'Hellenistic': 'green',
    'Roman': 'red',
    'Archaic': 'orange',
    'LC': 'purple',
    'Geometric': 'cadetblue',
    '': 'gray'
}

for _, row in gdf.iterrows():
    periode = row['Date'] if row['Date'] in couleurs_periodes else ''
    color = couleurs_periodes[periode]

    popup_text = f"""
    <div style="font-family: sans-serif;">
        <b>Language:</b> {row['Type']}<br>
        <b>Site:</b> {row['Site']}<br>
        <b>Nature:</b> {row['Nature']}<br>
        <b>Période:</b> {row['Date'] if row['Date'] != '' else 'N/A'}<br>
        <b>Précision:</b> {row['Precision_Date']}<br>
        <b>Source Geo:</b> {'Pleiades' if pd.notna(row['id_pleiades']) else 'Autre'}
    </div>
    """

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(marker_cluster)

m.save('carte_inscriptions_chypre.html')
print(f"{len(gdf)} inscriptions projetées sur la carte.")
m